# **Analiza statystyczna zbioru danych dot. ślimaków zwanych słuchotkami (lub uchowcami)**

**Importowanie pakietu pandas i numpy oraz wyświetlenie ich wersji**

In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display

print('\nWersja pakietu pandas: ', pd.__version__)
print('Wersja pakietu numpy: ', np.__version__)


Wersja pakietu pandas:  1.1.4
Wersja pakietu numpy:  1.21.3


**Importowanie danych z pliku csv oraz nadanie nazw kolumnom**

In [2]:
col_names = ['sex', 'length', 'diameter',
            'height', 'whole weight', 'shucked weight',
            'viscera weight', 'shell weight', 'rings']

data = pd.read_csv('data.csv', names = col_names)

*Wymagania na ocenę dostateczną:*

1. Tabela dla cechy jakościowej

In [3]:
count = data['sex'].value_counts()
percentage = []

for i in range (len(count)):
    percentage.append((count[i] / data['sex'].count())*100)

percentage = pd.Series(percentage, index = ['M', 'I', 'F'])

output = pd.concat([count, percentage], axis = 1)
output.columns = ['count', 'percentage']

display(output)

,count,percentage
M,1528,36.581278
I,1342,32.128322
F,1307,31.290400


2. Tabela dla cech ilościowych

